In [1]:
# Basic imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import scipy.optimize as spo
import sys
from time import time
from sklearn.metrics import r2_score, median_absolute_error

%matplotlib inline

%pylab inline
pylab.rcParams['figure.figsize'] = (20.0, 10.0)

%load_ext autoreload
%autoreload 2

sys.path.append('../../')
import predictor.feature_extraction as fe
import utils.preprocessing as pp
import utils.misc as misc

Populating the interactive namespace from numpy and matplotlib


In [2]:
best_params_df = pd.read_pickle('../../data/best_params_final_df.pkl')
best_params_df

,GOOD_DATA_RATIO,SAMPLES_GOOD_DATA_RATIO,ahead_days,base_days,model,mre,r2,step_days,train_days,train_val_time,x_filename,y_filename
ahead_days,,,,,,,,,,,,
1.0,0.99,0.9,1.0,112.0,linear,0.015856,0.986599,7.0,504.0,-1.0,x_base112_ahead1.pkl,y_base112_ahead1.pkl
7.0,0.99,0.9,7.0,112.0,linear,0.042367,0.923348,7.0,756.0,-1.0,x_base112_ahead7.pkl,y_base112_ahead7.pkl
14.0,0.99,0.9,14.0,112.0,linear,0.060167,0.865259,7.0,756.0,-1.0,x_base112_ahead14.pkl,y_base112_ahead14.pkl
28.0,0.99,0.9,28.0,112.0,linear,0.091966,0.758046,7.0,756.0,-1.0,x_base112_ahead28.pkl,y_base112_ahead28.pkl
56.0,0.99,0.9,56.0,112.0,linear,0.127913,0.590426,7.0,756.0,-1.0,x_base112_ahead56.pkl,y_base112_ahead56.pkl


## Ahead 1

### Without retraining

In [3]:
base_days = 112
ahead_days = 1
pid = 'base{}_ahead{}'.format(base_days, ahead_days)

x_train = pd.read_pickle('../../data/x_{}.pkl'.format(pid))
y_train = pd.read_pickle('../../data/y_{}.pkl'.format(pid))
x_test = pd.read_pickle('../../data/x_{}_test.pkl'.format(pid))
y_test = pd.read_pickle('../../data/y_{}_test.pkl'.format(pid))

In [4]:
end_date = x_train.index.levels[0][-1]
end_date

Timestamp('2014-07-18 00:00:00')

In [5]:
best_params_df.loc[ahead_days, 'train_days']

504.0

In [6]:
start_date = fe.add_market_days(end_date, -best_params_df.loc[ahead_days, 'train_days'])
start_date

Timestamp('2012-07-16 00:00:00')

In [9]:
x_sub_df = x_train.loc[(slice(start_date,None),slice(None)),:]
x_sub_df.shape

(20803, 112)

In [10]:
y_sub_df = y_train.loc[(slice(start_date,None),slice(None))]
y_sub_df.shape

(20803,)